In [135]:
import csv
import networkx as nx 
import numpy as np
import pandas as pd 
import community 
from itertools import compress
import matplotlib.pyplot as plt 
import seaborn as sns
from seaborn import color_palette, set_style, palplot
plt.style.use('ggplot')
%matplotlib inline

#### Load and clean data 

In [136]:
## read pre-processed data from stata
df = pd.read_stata('../data/0_CPIS_CDIS_BIS_USTIC_merged_fixed1.dta')
keep_var = ['countrycode','counterpart_code','country','counterpart','year','CDIS_IAD','CPIS_IAP','loans_dep']
df = df[keep_var]                           ## keep only used variables 
df = df.replace(np.nan,0)
num = df._get_numeric_data()
num[num < 0] = 0
df['total'] = df[['CDIS_IAD','CPIS_IAP','loans_dep']].sum(axis=1)
df.describe()

,countrycode,counterpart_code,year,CDIS_IAD,CPIS_IAP,loans_dep,total
count,102611.000000,102611.000000,102611.000000,1.026110e+05,1.026110e+05,1.026110e+05,1.026110e+05
mean,448.978973,459.162506,2011.827367,2.274913e+03,3.019251e+03,1.636379e+03,6.930741e+03
std,286.647552,296.652344,2.220511,2.192672e+04,3.130591e+04,2.060122e+04,6.117723e+04
min,111.000000,111.000000,2008.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,176.000000,176.000000,2010.000000,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00
50%,419.000000,423.000000,2012.000000,1.082549e-01,0.000000e+00,1.000000e+00,2.658900e+01
75%,666.000000,686.000000,2014.000000,4.144426e+01,3.022868e+01,4.900000e+01,3.830000e+02
max,968.000000,968.000000,2015.000000,1.115748e+06,1.816922e+06,1.288868e+06,2.767445e+06


In [137]:
mata = ['countrycode','counterpart_code','country','counterpart','year']
var_org = ['CDIS_IAD','CPIS_IAP','loans_dep','total']
var_sum_out = ['CDIS_Sum_out','CPIS_Sum_out','loans_dep_Sum_out','total_Sum_out']
var_sum_in = ['CDIS_Sum_in','CPIS_Sum_in','loans_dep_Sum_in','total_Sum_in']
var_weight = ['CDIS_weight','CPIS_weight','loans_dep_weight','total_weight']

df[var_sum_out]= df.groupby(['countrycode','year'])[var_org_out].transform(sum)           ## like stata egen sum 
df[var_sum_in]= df.groupby(['counterpart_code','year'])[var_org_in].transform(sum) 
df_weight = pd.DataFrame(df[var_org].values / df[var_sum].values,columns=[var_weight])
df[var_weight] = df_weight                                                        ## create the weight variables 
mata.extend(var_weight)
df = df[mata]
df.fillna(0,inplace=True)

/home/ubuntu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: RuntimeWarning: invalid value encountered in true_divide
